In [ ]:
import csv
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
import json

In [ ]:
base_path = "/home/zhihao/ChatGPT_analysis_latest/all_datasets/expression_match/chatgpt"

In [ ]:
def load_plm(model_name):
  # 确保使用的是AutoModelForMaskedLM来加载掩码语言模型
  tokenizer = AutoTokenizer.from_pretrained(f'/home/zhihao/.cache/huggingface/hub/{model_name}', trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(f'/home/zhihao/.cache/huggingface/hub/{model_name}', trust_remote_code=True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  return tokenizer, model

In [ ]:
tokenizer, model = load_plm('starchat-alpha')

In [ ]:
def get_answer(question):
  inputs = tokenizer.encode(question, return_tensors="pt").to(model.device)
  outputs = model.generate(inputs, max_length=2048)
  response = tokenizer.decode(outputs[0])

  return response

In [ ]:
import csv
import os

def create_csv_with_files(base_dirs, csv_filename):
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        filewriter = csv.writer(csvfile)
        filewriter.writerow(['filename', 'status'])  # 写入标题行

        for base_dir in base_dirs:
            for file in sorted(os.listdir(base_dir)):
                filewriter.writerow([base_dir + '/' + file, 0])  # 初始状态为0

types = sorted(os.listdir(base_path))
base_dirs = [os.path.join(base_path, f) for f in types]
csv_filename = 'expression_status.csv'
create_csv_with_files(base_dirs, csv_filename)


In [ ]:
import csv

def read_status_from_csv():
    status_dict = {}
    with open(csv_filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # 跳过标题行
        for row in reader:
            status_dict[row[0]] = int(row[1])  # 假设文件名在第一列，状态在第二列
    return status_dict


In [ ]:

def update_csv_status(filename, new_status):
    rows = []
    with open(csv_filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[0] == filename:
                row[1] = new_status
            rows.append(row)

    with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerows(rows)


In [ ]:
file_status = read_status_from_csv()
types = os.listdir(base_path)
types = sorted(types)
for f in types:
  first_path = os.path.join(base_path, f)
  files = os.listdir(first_path)
  files = sorted(files)
  for sub_f in files:
    question_dir = os.path.join(first_path, sub_f)
    print(question_dir)
    question_file = os.path.join(question_dir, "question_0.md")
    answer_file = os.path.join(question_dir, "starchat.md")

    # 检查该文件的状态
    if file_status.get(question_dir, 0) == 1:
        print(f"跳过已处理文件: {question_file}")
        continue

    with open(question_file, "r") as file:
        question = file.read()

    # 假设这是获取答案的函数
    answer = get_answer(question)  # 替换为您的答案获取逻辑

    with open(answer_file, "w") as file:
        file.write(answer)
    
    # 更新文件状态为1
    update_csv_status(question_dir, 1)

: 